# Deploy a reasoning LLM

<div align="left">
<a target="_blank" href="https://console.anyscale.com/template-preview/deployment-serve-llm?file=%252Ffiles%252Freasoning-llm"><img src="https://img.shields.io/badge/🚀 Run_on-Anyscale-9hf"></a>&nbsp;
<a href="https://github.com/ray-project/ray/tree/master/doc/source/serve/tutorials/deployment-serve-llm/reasoning-llm" role="button"><img src="https://img.shields.io/static/v1?label=&amp;message=View%20On%20GitHub&amp;color=586069&amp;logo=github&amp;labelColor=2f363d"></a>&nbsp;
</div>

A reasoning LLM handles tasks that require deeper analysis or step-by-step thought. It generates intermediate reasoning before arriving at a final answer, making it better suited for situations where careful logic or structured problem-solving is more important than speed or efficiency.

This tutorial deploys a reasoning LLM using Ray Serve LLM.  

---

## Compare reasoning and non-reasoning models

Reasoning models simulate step-by-step, structured thought processes to solve complex tasks like math, multi-hop QA, or code generation. In contrast, non-reasoning models provide fast, direct responses and focus on fluency or instruction following without explicit intermediate reasoning. The key distinction lies in whether the model attempts to "think through" the problem before answering.

| **Model type**          | **Core behavior**                    | **Use case examples**                                    | **Limitation**                                        |
| ----------------------- | ------------------------------------ | -------------------------------------------------------- | ----------------------------------------------------- |
| **Reasoning model**     | Explicit multi-step thinking process | Math, coding, logic puzzles, multi-hop QA, CoT prompting | Slower response time, more tokens used.                |
| **Non-reasoning model** | Direct answer generation             | Casual queries, short instructions, single-step answers  | May struggle with complex reasoning or interpretability. |

Many reasoning-capable models structure their outputs with special markers such as `<think>` tags, or expose reasoning traces inside dedicated fields like `reasoning_content` in the OpenAI API response. Always check the model's documentation for how to structure and control thinking.

**Note:** Reasoning LLMs often benefit from long context windows (32K up to +1M tokens), high token throughput, low-temperature decoding (greedy sampling), and strong instruction tuning or scratchpad-style reasoning.

---

### Choose when to use reasoning models

Whether you should use a reasoning model depends on how much information your prompt already provides.

If your input is clear and complete, a standard model is usually faster and more efficient. If your input is ambiguous or complex, a reasoning model works better because it can work through the problem step by step and fill in gaps through intermediate reasoning.

---

## Parse reasoning outputs

Reasoning models often separate *reasoning* from the *final answer* using tags like `<think>...</think>`. Without a proper parser, this reasoning may end up in the `content` field instead of the dedicated `reasoning_content` field.

To extract reasoning correctly, configure a `reasoning_parser` in your Ray Serve deployment. This tells vLLM how to isolate the model’s thought process from the rest of the output.  
**Note:** For example, *QwQ* uses the `deepseek-r1` parser. Other models may require different parsers. See the [vLLM docs](https://docs.vllm.ai/en/stable/features/reasoning_outputs.html#supported-models) or your model's documentation to find a supported parser, or [build your own](https://docs.vllm.ai/en/stable/features/reasoning_outputs.html#how-to-support-a-new-reasoning-model) if needed.

```yaml
applications:
- name: reasoning-llm-app
  ...
  args:
    llm_configs:
      - model_loading_config:
          model_id: my-qwq-32B
          model_source: Qwen/QwQ-32B
        ...
        engine_kwargs:
          ...
          reasoning_parser: deepseek_r1 # <-- for QwQ models
```

See [Configure Ray Serve LLM](#configure-ray-serve-llm) for a complete example.

**Example response**  
When using a reasoning parser, the response is typically structured like this:

```python
ChatCompletionMessage(
    content="The temperature is...",
    ...,
    reasoning_content="Okay, the user is asking for the temperature today and tomorrow..."
)
```
And you can extract the content and reasoning as follows:
```python
response = client.chat.completions.create(
  ...
)

print(f"Content: {response.choices[0].message.content}")
print(f"Reasoning: {response.choices[0].message.reasoning_content}")
```

---

## Configure Ray Serve LLM

Set your Hugging Face token in the config file to access gated models.

Ray Serve LLM provides multiple [Python APIs](https://docs.ray.io/en/latest/serve/api/index.html#llm-api) for defining your application. Use [`build_openai_app`](https://docs.ray.io/en/latest/serve/api/doc/ray.serve.llm.build_openai_app.html#ray.serve.llm.build_openai_app) to build a full application from your [`LLMConfig`](https://docs.ray.io/en/latest/serve/api/doc/ray.serve.llm.LLMConfig.html#ray.serve.llm.LLMConfig) object.

Set `tensor_parallel_size=8` to distribute the model's weights among 8 GPUs in the node. 

In [1]:
# serve_qwq_32b.py
from ray.serve.llm import LLMConfig, build_openai_app
import os

llm_config = LLMConfig(
    model_loading_config=dict(
        model_id="my-qwq-32B",
        model_source="Qwen/QwQ-32B",
    ),
    experimental_configs=dict(num_ingress_replicas=1),
    deployment_config=dict(
        # Increase number of replicas for higher throughput/concurrency.
        autoscaling_config=dict(
            min_replicas=1,
            max_replicas=1,
        )
    ),
    ### Uncomment if your model is gated and needs your Hugging Face token to access it
    # runtime_env=dict(env_vars={"HF_TOKEN": os.environ.get("HF_TOKEN")}),
    engine_kwargs=dict(
        # 4 GPUs is enough but you can increase tensor_parallel_size to fit larger models.
        tensor_parallel_size=4, max_model_len=32768, reasoning_parser="deepseek_r1"
    ),
)

app = build_openai_app({"llm_configs": [llm_config]})


INFO 02-06 10:57:39 [__init__.py:220] No platform detected, vLLM is running on UnspecifiedPlatform


2026-02-06 10:57:40,881	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 10.128.5.219:6379...
2026-02-06 10:57:40,892	INFO worker.py:2004 -- Connected to Ray cluster. View the dashboard at https://session-9fyy71sw3bgwajvnjflq7jxd9h.i.anyscaleuserdata.com 
2026-02-06 10:57:40,893	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_1172d88e13953c9d25fecc043fb49fa66a735377.zip' (0.03MiB) to Ray cluster...
2026-02-06 10:57:40,894	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_1172d88e13953c9d25fecc043fb49fa66a735377.zip'.
/home/ray/anaconda3/lib/python3.11/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
INFO 2026-02-06 10:57:40,903 serve 558853 -- ============== Deployment O

**Note:** Before moving to a production setup, migrate to a [Serve config file](https://docs.ray.io/en/latest/serve/production-guide/config.html) to make your deployment version-controlled, reproducible, and easier to maintain for CI/CD pipelines. See [Serving LLMs - Quickstart Examples: Production Guide](https://docs.ray.io/en/latest/serve/llm/quick-start.html#production-deployment) for an example.

---

## Deploy locally

**Prerequisites**

* Access to GPU compute.
* (Optional) A **Hugging Face token** if using gated models. Store it in `export HF_TOKEN=<YOUR-TOKEN-HERE>`

**Note:** Depending on the organization, you can usually request access on the model's Hugging Face page. For example, Meta’s Llama models approval can take anywhere from a few hours to several weeks.

**Dependencies:**  
```bash
pip install "ray[serve,llm]"
```

---

### Launch the service

Follow the instructions at [Configure Ray Serve LLM](#configure-ray-serve-llm) to define your app in a Python module `serve_qwq_32b.py`.  

In a terminal, run:  

In [4]:
!serve run serve_qwq_32b:app --non-blocking

2026-02-06 10:58:54,882	INFO scripts.py:507 -- Running import path: 'serve_qwq_32b:app'.
INFO 02-06 10:58:57 [__init__.py:220] No platform detected, vLLM is running on UnspecifiedPlatform
2026-02-06 10:58:58,670	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 10.128.5.219:6379...
2026-02-06 10:58:58,680	INFO worker.py:2004 -- Connected to Ray cluster. View the dashboard at https://session-9fyy71sw3bgwajvnjflq7jxd9h.i.anyscaleuserdata.com 
2026-02-06 10:58:58,681	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_390354c9d595384f599abb432b5ac4098be20a2f.zip' (0.03MiB) to Ray cluster...
2026-02-06 10:58:58,682	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_390354c9d595384f599abb432b5ac4098be20a2f.zip'.
/home/ray/anaconda3/lib/python3.11/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default).

Deployment typically takes a few minutes as the cluster is provisioned, the vLLM server starts, and the model is downloaded. 

---

### Send requests

Your endpoint is available locally at `http://localhost:8000` and you can use a placeholder authentication token for the OpenAI client, for example `"FAKE_KEY"`.

Example curl:

In [5]:
!curl -X POST http://localhost:8000/v1/chat/completions \
  -H "Authorization: Bearer FAKE_KEY" \
  -H "Content-Type: application/json" \
  -d '{ "model": "my-qwq-32B", "messages": [{"role": "user", "content": "Pick three random words with 3 syllables each and count the number of R'\''s in each of them"}] }'

{"id":"chatcmpl-571cfac9-de14-48ed-b7f9-103e70493939","object":"chat.completion","created":1770404787,"model":"my-qwq-32B","choices":[{"index":0,"message":{"role":"assistant","content":"\n\nThe three random words with three syllables each and the number of R's in each are:\n\n1. **Adventure**  \n   - Syllables: Ad-ven-ture (3)  \n   - Number of R's: 1  \n\n2. **Tomorrow**  \n   - Syllables: To-mor-row (3)  \n   - Number of R's: 2  \n\n3. **Paragraph**  \n   - Syllables: Par-a-graph (3)  \n   - Number of R's: 2  \n\n**Final Answer**  \nThe number of R's in the words are:  \n1. Adventure: \\boxed{1}  \n2. Tomorrow: \\boxed{2}  \n3. Paragraph: \\boxed{2}","refusal":null,"annotations":null,"audio":null,"function_call":null,"tool_calls":[],"reasoning_content":"Okay, the user wants me to pick three random words with three syllables each and then count the number of R's in each. Let me start by recalling what a syllable is. A syllable is a unit of pronunciation having one vowel sound, so thre

Example Python:

In [6]:
#client.py
from urllib.parse import urljoin
from openai import OpenAI

API_KEY = "FAKE_KEY"
BASE_URL = "http://localhost:8000"

client = OpenAI(base_url=urljoin(BASE_URL, "v1"), api_key=API_KEY)

response = client.chat.completions.create(
    model="my-qwq-32B",
    messages=[
        {"role": "user", "content": "What is the sum of all even numbers between 1 and 100?"}
    ]
)

print(f"Reasoning: \n{response.choices[0].message.reasoning_content}\n\n")
print(f"Answer: \n {response.choices[0].message.content}")

{"asctime": "2026-02-06 11:10:51,339", "levelname": "INFO", "message": "HTTP Request: POST http://localhost:8000/v1/chat/completions \"HTTP/1.1 200 OK\"", "filename": "_client.py", "lineno": 1025, "process": 558853, "job_id": "06000000", "worker_id": "06000000ffffffffffffffffffffffffffffffffffffffffffffffff", "node_id": "1a6ddbbb716b74256e415b58e3dca445abdb4074bbfecbc482406ab0", "timestamp_ns": 1770405051339564012}


Reasoning: 
Okay, so I need to find the sum of all even numbers between 1 and 100. Hmm, let me think. First, I guess I should clarify what exactly the question is asking. When it says "between 1 and 100," does it include 1 and 100 themselves? Because sometimes "between" can be a bit ambiguous. Well, 1 is odd, so whether it's included or not doesn't affect the sum of even numbers. But 100 is even, so if it's included, we need to add it in. Let me check the problem again. It says "between 1 and 100." Hmm, maybe it's safer to assume it's exclusive? Wait, but in math problems like this, sometimes they mean from 1 up to and including 100. Let me see. Well, even if it's exclusive, 100 is the upper limit, but since 100 is even, maybe the question expects it to be included. Maybe I should confirm that. Alternatively, maybe I can just calculate both cases, but first, let me see if there's a standard interpretation here.

Alternatively, perhaps the problem is straightforward, and they just want 

If you configure a valid reasoning parser, the reasoning output should appear in the `reasoning_content` field of the response message. Otherwise, it may be included in the main `content` field, typically wrapped in `<think>...</think>` tags. See [Parse reasoning outputs](#parse-reasoning-outputs) for more information.

---

### Shutdown

Shutdown your LLM service:

In [ ]:
!serve shutdown -y

2026-02-06 11:12:18,732	SUCC scripts.py:774 -- Sent shutdown request; applications will be deleted asynchronously.


(autoscaler +16m53s) [autoscaler] There are insufficient compute resources to run this workload. Common reasons include (a) not adding or configuring node groups with sufficient resources, (b) requesting a resource (e.g. accelerator) that is not available in the region/provider of your Anyscale cloud, or (c) use of certain Ray scheduling strategies, e.g. STRICT_SPREAD, in a way that doesn't allow resources to fit on a node. Requests:
1 * {"resourcesBundle":{"CPU":1, "GPU":1, "accelerator_type:L40S":0.001}}, 3 * {"resourcesBundle":{"GPU":1, "accelerator_type:L40S":0.001}}, strategy='STRICT_PACK'



---

## Deploy to production with Anyscale services

For production, use Anyscale services to deploy your Ray Serve app on a dedicated cluster without code changes. Anyscale provides scalability, fault tolerance, and load balancing, ensuring resilience against node failures, high traffic, and rolling updates. See [Deploy a medium-sized LLM](https://docs.ray.io/en/latest/serve/tutorials/deployment-serve-llm/medium-size-llm/README.html#deploy-to-production-with-anyscale-services) for an example with a medium-sized model like the *QwQ-32&nbsp;B* used here.

---

## Stream reasoning content

Reasoning models may take longer to begin generating the main content. You can stream their intermediate reasoning output in the same way as the main content.  

In [7]:
#client_streaming.py
from urllib.parse import urljoin
from openai import OpenAI

API_KEY = "FAKE_KEY"
BASE_URL = "http://localhost:8000"

client = OpenAI(base_url=urljoin(BASE_URL, "v1"), api_key=API_KEY)

# Example: Complex query with thinking process
response = client.chat.completions.create(
    model="my-qwq-32B",
    messages=[
        {"role": "user", "content": "I need to plan a trip to Paris from Seattle. Can you help me research flight costs, create an itinerary for 3 days, and suggest restaurants based on my dietary restrictions (vegetarian)?"}
    ],
    stream=True
)

# Stream
for chunk in response:
    # Stream reasoning content
    if hasattr(chunk.choices[0].delta, "reasoning_content"):
        data_reasoning = chunk.choices[0].delta.reasoning_content
        if data_reasoning:
            print(data_reasoning, end="", flush=True)
    # Later, stream the final answer
    if hasattr(chunk.choices[0].delta, "content"):
        data_content = chunk.choices[0].delta.content
        if data_content:
            print(data_content, end="", flush=True)

{"asctime": "2026-02-06 11:11:28,785", "levelname": "INFO", "message": "HTTP Request: POST http://localhost:8000/v1/chat/completions \"HTTP/1.1 200 OK\"", "filename": "_client.py", "lineno": 1025, "process": 558853, "job_id": "06000000", "worker_id": "06000000ffffffffffffffffffffffffffffffffffffffffffffffff", "node_id": "1a6ddbbb716b74256e415b58e3dca445abdb4074bbfecbc482406ab0", "timestamp_ns": 1770405088785791077}


Okay, the user wants to plan a trip from Seattle to Paris for three days, focusing on flight costs, an itinerary, and vegetarian restaurants. Let me start by breaking down each part.

First, flights. I need to check current flight prices from Seattle-Tacoma (SEA) to Paris-Charles de Gaulle (CDG). Since the user didn't specify dates, I'll assume they might be looking for a general idea, so I'll mention that prices can vary and suggest checking specific dates. I'll note that budget airlines like Air France or Delta might have deals, and maybe mention average costs, like around $800-1200 round trip. Also, highlight booking in advance and flexible dates.

Next, the itinerary. Three days in Paris is a short trip, so I should focus on the most iconic spots. Day 1 could start with the Eiffel Tower, then maybe the Seine River cruise, and end at Montmartre. Day 2 might include the Louvre and Notre-Dame, lunch in Le Marais, and evening at Champs-Élysées. Day 3 could be Versailles, but since it's


---

## Summary

In this tutorial, you deployed a reasoning LLM with Ray Serve LLM, from development to production. You learned how to configure Ray Serve LLM with the right reasoning parser, deploy your service on your Ray cluster, send requests, and parse reasoning outputs in the response.